# Regression with `sklearn`

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression

## Data Setup

In [2]:
wine = pd.read_csv('../data/wine.csv')

wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [3]:
wine_preds = wine.drop('quality', axis=1)
wine_target = wine['quality']

## Scale the Data

In [4]:
wine_preds_scaled = (wine_preds - wine_preds.mean()) / wine_preds.std(ddof=0)

In [6]:
# Let's create a StandardScaler object to scale our data for us.
ss = StandardScaler()

In [7]:
# Now we'll apply it to our data by using the .fit() and .transform() methods.
ss.fit(wine_preds)

StandardScaler()

In [8]:
wine_preds_st_scaled = ss.transform(wine_preds)

wine_preds_st_scaled

array([[ 0.14247327,  2.18883292, -2.19283252, ...,  0.19309677,
        -0.91546416,  1.75018984],
       [ 0.45103572,  3.28223494, -2.19283252, ...,  0.99957862,
        -0.58006813,  1.75018984],
       [ 0.45103572,  2.55330026, -1.91755268, ...,  0.79795816,
        -0.58006813,  1.75018984],
       ...,
       [-0.55179227, -0.6054167 , -0.88525328, ..., -0.47897144,
        -0.91546416, -0.57136659],
       [-1.32319841, -0.30169391, -0.12823371, ..., -1.016626  ,
         1.9354021 , -0.57136659],
       [-0.93749534, -0.78765037,  0.42232597, ..., -1.41986693,
         1.09691202, -0.57136659]])

In [9]:
ss.fit_transform(wine_preds)

array([[ 0.14247327,  2.18883292, -2.19283252, ...,  0.19309677,
        -0.91546416,  1.75018984],
       [ 0.45103572,  3.28223494, -2.19283252, ...,  0.99957862,
        -0.58006813,  1.75018984],
       [ 0.45103572,  2.55330026, -1.91755268, ...,  0.79795816,
        -0.58006813,  1.75018984],
       ...,
       [-0.55179227, -0.6054167 , -0.88525328, ..., -0.47897144,
        -0.91546416, -0.57136659],
       [-1.32319841, -0.30169391, -0.12823371, ..., -1.016626  ,
         1.9354021 , -0.57136659],
       [-0.93749534, -0.78765037,  0.42232597, ..., -1.41986693,
         1.09691202, -0.57136659]])

In [ ]:
# Check that the scaling worked about the same as when we did it by hand
np.allclose(wine_preds_st_scaled, wine_preds_scaled)

In [ ]:
wine_preds_scaled.head()

In [ ]:
wine_preds_st_scaled[:5, :]

## Fit the Model

Now we can fit a `LinearRegression` object to our training data!

In [10]:
# Now we can fit a LinearRegression object to our training data!

lr = LinearRegression()
lr.fit(wine_preds_st_scaled, wine_target)

LinearRegression()

In [12]:
wine_preds.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'red_wine'],
      dtype='object')

In [11]:
# We can use the .coef_ attribute to recover the results
# of the regression.

lr.coef_

array([ 0.11027401, -0.24568548, -0.00909927,  0.29704168, -0.02652718,
        0.08762284, -0.07927578, -0.311567  ,  0.08018737,  0.10739154,
        0.26556038,  0.155642  ])

In [13]:
pd.DataFrame(lr.coef_, wine_preds.columns)

,0
fixed acidity,0.110274
volatile acidity,-0.245685
citric acid,-0.009099
residual sugar,0.297042
chlorides,-0.026527
free sulfur dioxide,0.087623
total sulfur dioxide,-0.079276
density,-0.311567
pH,0.080187
sulphates,0.107392


In [ ]:
lr.intercept_

In [ ]:
lr.score(wine_preds_st_scaled, wine_target)

In [ ]:
y_hat = lr.predict(wine_preds_st_scaled)
y_hat

All that's left is to evaluate our model to see how well it did!

## Evaluate Performance

### Observing Residuals

We can check the residuals like we would for a simple linear regression model.

In [ ]:
y_hat = lr.predict(wine_preds_st_scaled)
resid = (wine_target - y_hat)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x=range(y_hat.shape[0]),y=resid, alpha=0.1);

### Sklearn Metrics

The metrics module in sklearn has a number of metrics that we can use to measure the accuracy of our model, including the $R^2$ score, the mean absolute error and the mean squared error. Note that the default 'score' on our model object is the $R^2$ score. Let's go back to our wine dataset:

In [14]:
metrics.r2_score(wine_target, lr.predict(wine_preds_st_scaled))

0.29653465192890527

Let's make sure this metric is properly calibrated. If we put simply $\bar{y}$ as our prediction, then we should get an $R^2$ score of *0*. And if we predict, say, $\bar{y} + 1$, then we should get a *negative* $R^2$ score.

In [ ]:
avg_quality = np.mean(wine_target)
num = len(wine_target)

In [ ]:
metrics.r2_score(wine_target, avg_quality * np.ones(num))

In [ ]:
metrics.r2_score(wine_target, (avg_quality + 1) * np.ones(num))

In [15]:
metrics.mean_absolute_error(wine_target, lr.predict(wine_preds_st_scaled))

0.568537539040229

In [17]:
metrics.mean_squared_error(wine_target, lr.predict(wine_preds_st_scaled), squared=False)

0.7323676381358274

# Level Up: Deeper Evaluation of Wine Data Predictions

One thing we could have investigated from our [model on the Wine Data](#Multiple-Regression-in-Scikit-Learn) is how our predictions $\hat{y}$ match with the actual target values.

In [ ]:
sns.histplot(y_hat,kde=True,fill=False,stat='density',color='red')
sns.histplot(wine_target,discrete=True,stat='density')

So there's a slight issue with our model; the linear regression believes the target values are on a continuum. We know that's not true from the data. 

An easy fix is to round the target values.

In [ ]:
y_hat_rounded = np.round(y_hat)
np.unique(y_hat_rounded, return_counts=True)

In [ ]:
metrics.mean_squared_error(wine_target, y_hat_rounded)

Plotting the distribution is a lot more meaningful if we require targets to be integers.

In [ ]:
sns.histplot(np.round(y_hat),fill=False,discrete=True,stat='density',color='red')
sns.histplot(wine_target,discrete=True,alpha=0.3,stat='density')

Note that our $R^2$ metric will be worse. This makes sense since we found a "line of best fit" that predicts continuous values. 

If the better option was _integer_ predictions, it would have predicted that instead. 

In [ ]:
metrics.r2_score(wine_target, y_hat_rounded)

You must decide yourself if this is worth doing or if a different model makes more sense (we'll see more models in future lectures).

# Level Up: Regression with Categorical Features with the Comma Dataset

In [ ]:
commas = pd.read_csv('../data/comma-survey.csv')

In [ ]:
commas.head()

In [ ]:
ohe = OneHotEncoder(drop='first').fit(comma_df.drop('RespondentID', axis=1))

In [ ]:
comma_df = pd.DataFrame(ohe.transform(comma_df.drop('RespondentID', axis=1)).todense(),
                       columns=ohe.get_feature_names())

In [ ]:
comma_df.columns

In [ ]:
# We'll try to predict the first column of df: the extent to which
# the person accepts the sentence
# without the Oxford comma as more grammatically correct.

comma_target = comma_df['x0_It\'s important for a person to be honest, kind, and loyal.']

comma_predictors = comma_df[['x8_30-44',
       'x8_45-60', 'x8_> 60', 'x9_$100,000 - $149,999',
       'x9_$150,000+', 'x9_$25,000 - $49,999', 'x9_$50,000 - $99,999']]

comma_lr = LinearRegression()

comma_lr.fit(comma_predictors, comma_target)

comma_lr.score(comma_predictors, comma_target)

In [ ]:
comma_lr.coef_

In [ ]:
comma_df.corr()['x0_It\'s important for a person to be honest, kind, and loyal.']